<a href="https://colab.research.google.com/github/MatheoCruz/BLR_imputation_perovskite/blob/main/Proyecto_de_grado_imputacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *PROYECTO DE GRADO: ETAPA DE INFERENCIA BAYESIANA (Imputación)*


## IMPORTACION

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Importacion de librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymc as pm
import arviz as az
from functools import partial
from scipy.stats import norm
from sklearn.model_selection import KFold
from scipy import stats
import seaborn as sns
import pickle
#Importacion de base de datos
#Import data base
data = pd.DataFrame(pd.read_csv("/content/drive/MyDrive/Deep Learning/TG/Preprocesamiento/datos_sintesis.csv"))#datos de entrenamiento preprocesados
priori_x_grids = pd.DataFrame(pd.read_csv("/content/drive/MyDrive/Deep Learning/TG/Preprocesamiento/priori_x_grids.csv")) #grilla eje X del PDF priori generada por KDE para las variables A, B, X, Band_gap, Thickness, PCE, Jsc, FF, Voc
priori_y_grids = pd.DataFrame(pd.read_csv("/content/drive/MyDrive/Deep Learning/TG/Preprocesamiento/priori_y_grids.csv")) #grilla eje Y del PDF priori generada por KDE para las variables A, B, X, Band_gap, Thickness, PCE, Jsc, FF, Voc

In [4]:
#creamos los kde para la construccion de los priori
priori_x_grids = {'A':priori_x_grids['A'], 'B':priori_x_grids['B'], 'X':priori_x_grids['X'], 'Band_gap':priori_x_grids['Bandgap'],
         'Thickness':priori_x_grids['Thickness'], 'PCE':priori_x_grids['PCE'], 'Voc':priori_x_grids['Voc'], 'Jsc':priori_x_grids['Jsc'], 'FF':priori_x_grids['FF']}
priori_y_grids = {'A':priori_y_grids['A'], 'B':priori_y_grids['B'], 'X':priori_y_grids['X'], 'Band_gap':priori_y_grids['Bandgap'],
         'Thickness':priori_y_grids['Thickness'], 'PCE':priori_y_grids['PCE'], 'Voc':priori_y_grids['Voc'], 'Jsc':priori_y_grids['Jsc'], 'FF':priori_y_grids['FF']}

## FUNCIONES
### correlacion_pearson:
Esta funcion calculo la correlacion de pearson existente entre las variables del modelo y sus posibles transformaciones, dotando esta relacion de un valor numerico.
### transformation_optima:
Esta funcion toma como entrada la correlacion existente entre las variables y sus posibles transformaciones y busca obtener la mejor transformacion dado el valor numerico de la correlacion y un umbral minimo que ha de cumplir dicha correlacion.
### transformation_by_variable:
Toma como parametro de entrada la transformacion optima, y dado una variable de interes se ordenan para esta las transformaciones de cada variable asociada a esta.
### make_transformations:
Esta funcion toma un conjunto de datos y una variable de interes, a la que dada las transformaciones optimas por variable realiza las transformaciones indicadas al conjunto de datos.
### pdf_normal:
Retorna una funcion de densidad de probabilidad normal, asociandola a un nombre, una media y una desviacion estandar.
### build_linear_model_bayesian
$$Y = βX + α$$
Tomando como comparativa la regresion lineal tradicional, la regresion lineal bayesiana es muy similar, siendo $α$ la funcion de densidad de probabiliad priori, es decir la creencia previa del comportamiento de $Y$, y siendo $\beta$ los parametros predictores asociados a las entradas $X$ del modelo.

In [5]:
#FUNCIONES
#Constructor de una PDF normal
def pdf_normal(name, mean, std):
  return pm.Normal(name, mu = mean, sigma = std) #Retorna una PDF normal con una clave o etiqueta dada por "name".
#-------------------------------------------------------------------------------

#Funcion de normalizacion de PDF
def normalizacion_pdf(grid_x, grid_y):
  #Escalar la distribucion para que el area bajo la curva sea igual a 1
  area_bajo_curva = np.trapz(grid_y, grid_x)
  grid_y_normalizada = grid_y/area_bajo_curva

  #Calcular el promedio ponderado de grid_x para centrar en cero
  media_ponderada = np.trapz(grid_x * grid_y_normalizada, grid_x)
  grid_x_centro_cero = grid_x - media_ponderada

  return grid_x_centro_cero, grid_y_normalizada
#-------------------------------------------------------------------------------

#Funcion de correlacion de pearson
def correlation_pearson(data):
  transformations = [None, np.log, np.exp, np.square, np.sqrt] #Transformaciones u operaciones predeterminadas a analizar.
  name_transformations = ["None", "log", "exp", "square", "sqrt"] #String representativo de las transformaciones.
  correlation_types = ["pearson"] #Se define el tipo de correlacion que se va determinar "pearson" en este caso, indica la correlacioon directamente lineal.
  # Calcular correlaciones
  correlations = {} #Variable de salida que contiene la correlacion de cada variable frente a las otras variables.
  restrictions = {} #Variable de salida que contiene las variables cuya transformacion genera indeterminaciones.
  for column_name in data.columns: #Se extrae columna por columna su determiando indice para realizar el analisis.
      column = data[column_name] #Se extrae dicha columna.
      column_correlations = {} #Variable temporal de la correlacion por columna.
      temporary_restrictions = {} #Variable temporalñ de las restricciones.
      i=0
      for transform in transformations: #Trasformaciones
          column_transformed = column if transform is None else column.apply(transform) #Se aplica la transformacion.
          for correlation_type in correlation_types:
              data_transformed = pd.concat([data.drop(column_name, axis=1), column_transformed], axis=1) #Columna de variable transformada.
              corr = data_transformed.corr(method=correlation_type) #Correlacion de dicha variable transformada con el resto de variables o columnas.
              column_correlations[name_transformations[i]] = corr[column_name][:-1] #Se guarda la correlacion dada la transformacion.
              if np.isnan(column_transformed).any(): #Condicionamiento de la restriccion
                temporary_restrictions[name_transformations[i]] = True
              else:
                temporary_restrictions[name_transformations[i]] = False
          i=i+1
          correlations[column_name] = column_correlations #Correlaciones por variable.
          restrictions[column_name] = temporary_restrictions #Restricciones por variable.
  return correlations, restrictions
#-------------------------------------------------------------------------------

#Funcion que determina la transformacion optima de la variable basado en la correlacion de pearson
def transformation_optima(correlations,restrictions,umbral = 0.55):
  correlations = pd.DataFrame(correlations) #Se convierte a DF el diccionario de correlaciones.
  m,n = correlations.shape #Se extraen sus dimensiones.
  optimal_transformation = {} #Variable de salida que contiene la transformacion optima
  transformations = [None, np.log, np.exp, np.square, np.sqrt] #Transformaciones
  name_transformations = ["None", "log", "exp", "square", "sqrt"] #String asociado a las transformaciones
  for columns in correlations.columns: #Se extrae variable por columna del diccionario de correlaciones.
    temporary_transformation = ["None"]*(n-1) #Variable temporal e inicial de la transformacion optima
    temporary_conditional = [0]*(n-1) #Variable de condicional temporal e inicial.
    for names in name_transformations: #Se analiza una transformacion a la vez.
      i = 0
      temporary_optimal_transformation = {} #variable temporal de las entradas de las transformaciones
      for i in range(n-1):
        key = correlations[columns][names].keys()[i] #variable correlacionada.
        conditional = round(correlations[columns][names][key],2) #Se establece la condicion de optima correlacion
        if restrictions[columns][names]: #Restriccion de indeterminaciones
          temporary_optimal_transformation[key] = 'None'
        else:
          if abs(conditional) > umbral: #Condicion de valor minimo absoluto de correlacion.
            if abs(conditional) > temporary_conditional[i]: #Condicion de valor mayor que el valor anterior de correlacion
              temporary_optimal_transformation[key] = names #Optima transformacion por variable correlacionada actual.
              temporary_transformation[i] = names #Se guarda en una variable temporal el nombre de la transformacion
              temporary_conditional[i] = abs(conditional) #Se guarda el valor actual de correlacion.
            else:
              temporary_optimal_transformation[key] = temporary_transformation[i] #Optima transfoprmacion por variable correlacionada anterior.
          else:
            temporary_optimal_transformation[key] = temporary_transformation[i] #Optima transformacion por variable correlacionada anterior o inicial.
    optimal_transformation[columns] = temporary_optimal_transformation #Optima transformacion pro variable.
  return optimal_transformation
#--------------------------------------------------------------------------------

#Funcion que construlle la transformacion optima de los parametros de entrada para una variable de interes
def transformation_by_variable(optimal_transformation):
  transformations_by_variable = {} #Se ordenal las transformaciones optimas que se realizaran dependiendo de la variable de interes.
  for keys in optimal_transformation.keys(): #keys del diccionario de transformaciones optimas por variable.
    variables = list(optimal_transformation.keys()) #Se convierte en una lista las keys
    variables.remove(keys) #Se remueve las variable actual
    temporary_transformation = {}
    for vars in variables: #Variables de interes
      temporary_transformation[vars] = optimal_transformation[vars][keys] # se guarda la transformacion de la variable igual a la key en la variable de interes actual.
    transformations_by_variable[keys] = temporary_transformation #Se guardan todas las transformaciones de las variables relacionadas de la variable de interes.
  return transformations_by_variable
#--------------------------------------------------------------------------------------

#Funcion que realiza las transformaciones de las entradas del modelo basados en su transformacion mas optima para la variable de interes
def make_transformations(data, condicion):
  output = data[condicion] #De un dataset de datos se extrae la variable de interes "variable de salida o variable a imputar".
  inputs = data.drop(condicion, axis = 1) #Variable relacionadas a la variable de interes o "variables de entrada".
  Correlations, Restrictions = correlation_pearson(data)#Se generan las correlaciones y sus posibles restricciones por transformacion.
  optimal_transformation = transformation_optima(Correlations, Restrictions, 0.55) #Se escogen las transformaciones optima con un umbral de 0.55.
  Transformations = transformation_by_variable(optimal_transformation)#Se ordenan las transformaciones dada la variable de interes es decir "output".
  dict_transformation = dict(zip(["None", "log", "exp", "square", "sqrt"], [None, np.log, np.exp, np.square, np.sqrt])) #Diccionario de transformaciones
  retorno = {} #variable de retorno de las entradas transformadas.
  for name_var in inputs.columns: #Se recorren las variables de entrada
    var = inputs[name_var]
    associated_transformation = dict_transformation[Transformations[condicion][name_var]] #Se asocia la variable de entrada a una transformacion optima por variable.
    var_transformada = var if associated_transformation is None else var.apply(associated_transformation) # Se realiza la transformacion.
    retorno[name_var] = var_transformada #Se retorna la variable transformada
  inputs = pd.DataFrame(retorno)
  return output, inputs
#--------------------------------------------------------------------------------------

#Funcion constructora del modelo
def build_linear_model_bayesian(X,y,data,entradas,priori_x_grids,priori_y_grids):
    with pm.Model() as modelo: #Se declara un elemento modelo de Pymc
        x_shared = pm.Data("x_shared", X, mutable = True)
        #Condicion para las variables que no cuentan con un Priori construido por medio de un KDE, se les genera una distribucion uniforme basada en el reango de valores en el que puede variar la variable
        if data == 'DeltaL':
          alpha = pm.Uniform(data, lower = -0.5, upper = 0.5) #Distribucion uniforme entre 0 y 1
        elif data == 'DeltaH':
          alpha = pm.Uniform(data, lower = -0.5, upper = 0.5) #Distribucion uniforme entre 0 y 1
        elif data == 'Grain_size':
          alpha = pm.Uniform(data, lower = -1.75, upper=  1.75) #Distribucion uniforme entre 0.01 y 3.5
        else:
          grid_x, grid_y = normalizacion_pdf(priori_x_grids[datos],priori_y_grids[datos])
          alpha = pm.Interpolated(data, np.asarray(grid_x), np.asarray(grid_y)) # "alpha" PDF asociada a la variables de salida "PDF"
        betas = [] #lista que contendra las PDF asociadas a las entradas.
        for entrada in entradas:
         betas.append(pdf_normal(entrada, 0, 10)) #PDF asociada a las variables de entrada del modelo
        sigma = pm.HalfNormal('sigma',sigma = 1)# Varianza del error
        y_est = pm.Deterministic('y_est', alpha + pm.math.dot(x_shared,betas)) #Modelo lineal
        likelihood = pm.Normal('y_obs',mu = y_est, sigma=sigma, observed = y) #Probabilidad asociada al modelo lineal
        # Regularización L2
        reg = pm.math.sum(pm.math.sqr(betas)) + pm.math.sqr(alpha)
        ridge = pm.Potential('ridge', -0.5*reg)
    return modelo
#-----------------------------------------------------------------------------------

## Resultados de correlacion y transformaciones.
En este apartado se ejemplifica el criterio tomado para realizar las transformaciones de las entradas del modelo basado en la correlacion de pearson, con intencion de no extender la explicacion se hara exclusivamente para "PCE".

In [6]:
correlations, restrictions = correlation_pearson(data) # Correlaciones y restricciones generadas por indeterminaciones.
optimal_transformation = transformation_optima(correlations,restrictions,umbral = 0.55) # Transformacion optima
transformations_by_variable = transformation_by_variable(optimal_transformation) # Transformaciones optimas para las entradas de cada variable de interes
name_transformations = ["None", "log", "exp", "square", "sqrt"] # Transformaciones
var_interes = "FF" #Variable de interes
correlaciones = {}
print(f"-----------------------------------{var_interes}--------------------------------------")
for datos in data.drop(var_interes, axis=1):
  correlacion = []
  for names in name_transformations:
    if restrictions[datos][names]: # Condicion de restriccion por indeterminacion
      correlacion_temporal = np.nan
    else:
      correlacion_temporal = correlations[datos][names][var_interes]
    correlacion.append(correlacion_temporal)
  correlaciones[datos] = correlacion
df_correlaciones = pd.DataFrame(correlaciones, index = name_transformations)
print(df_correlaciones)
print("----------Transformaciones optimas para cada variable-----------")
print(transformations_by_variable["PCE"])

-----------------------------------FF--------------------------------------
               A         B         X  Band_gap    DeltaH    DeltaL  \
None    0.010242 -0.048297  0.030050 -0.039296 -0.089970  0.166043   
log          NaN       NaN       NaN -0.043113 -0.194781  0.102890   
exp     0.012391 -0.048297  0.037926 -0.028678 -0.099887  0.154198   
square  0.003087 -0.048297  0.067611 -0.034323 -0.115098  0.115004   
sqrt         NaN -0.048297       NaN -0.041352 -0.017683  0.237509   

        Grain_size  Thickness       PCE       Voc       Jsc  
None      0.288337   0.367698  0.543788  0.121598  0.256438  
log       0.280713   0.303488  0.488240  0.110537  0.227533  
exp       0.265032   0.357654  0.414718  0.129298  0.058548  
square    0.267202   0.341567  0.575151  0.129972  0.268986  
sqrt      0.291124   0.354600  0.518853  0.116381  0.243638  
----------Transformaciones optimas para cada variable-----------
{'A': 'None', 'B': 'None', 'X': 'None', 'Band_gap': 'None', 'Delta

## CONSTRUCCION DEL MODELO
La etapa de construccion del modelo consta de varios pasos:
- Transformaciones de variables:

  Como se explico anteriormente el proceso que sigue este, es la extraccion de correlaciones para realizar transformaciones optimas de las variables del modelo en relacion a la variable a imputar.
- Segmentacion de los datos:

  Este proceso consiste en separar los conjuntos de datos de entrenamiento y testeo, esto se hace por medio de la funcion Kfold, que permite realizar un validacion cruzada, obteniendo de esta 5 disitintos modelos, es decir un 5-fold, esto da como resultado una proporcion 80% a 20% entre los datos de entrenamiento y testeo.
- Normalizacion de los datos:

  Se genera una normalizacion de los datos de entrenamiento, con la intencion del que modelo converja a una solucion mas optima en un menor tiempo.
- Construccion del modelo:

  Esto se realiza por medio de la funcion anteriormente construida "Restrictionsbuild_linear_model_bayesian". Que permite la construccion de un modelo basado en la variable de interes tomando como parametros, dicha variable de interes, sus variables de entrada transformadas asociadas, y el prior de dicha variable de interes.

Finalmente se hace un guardado de los modelos y los datos asociados a este para su posterior entrenamiento.

In [ ]:
#---------Definicion de los parametros de segmentacion--------------
k=5 # 5-fold
kf = KFold(n_splits = k)
#-------------------------------------------------------------------
#variables de guardar los modelos definidos
modelos = {} #save del modelo
data_train = {} #save de datos de entrenamiento
data_test = {} #save de datos de testeo
for datos in data[data.columns[3:]]: # For que permite abarcar todas las variables de interes a imputar
#---------------------------------Transformaciones---------------------------------------------------
  output, inputs = make_transformations(data, datos) # tranformacion por variable
#----------------------------------------------------------------------------------------------------
  #save temporales
  modelos_fold = {}
  data_train_fold = {}
  data_test_fold = {}
  trazados_fold = {}
  j=1
  print(datos)
#--------------------------------Segmentacion de datos----------------------------------------------
  for train_index, test_index in kf.split(inputs): # se genera el fold
    print('Fold #',j,'para el parametro', datos)
    x_train, x_test = np.asarray(inputs)[train_index], np.asarray(inputs)[test_index] #particiones para x (entradas)
    y_train, y_test = np.asarray(output)[train_index], np.asarray(output)[test_index] #particiones para y (salida)
    print(x_train.shape)
#---------------------------------------------------------------------------------------------------
#-------------------------------------Normalizacion-------------------------------------------------
    #Normalizacion
    x_mean = np.mean(x_train, axis = 0)
    x_std = np.std(x_train, axis = 0)
    y_mean = np.mean(y_train)
    y_std = np.std(y_train)
    X_train = (x_train-x_mean)/(x_std)
    Y_train = (y_train-y_mean)/(y_std)
    X_train[:,1] = x_train[:,1]
    X_test = (x_test-x_mean)/(x_std)
    X_test[:,1] = x_test[:,1]
    Y_test = (y_test-y_mean)/(y_std)
#---------------------------------------------------------------------------------------------------
#--------------------------------Construccion del modelo--------------------------------------------
    model = build_linear_model_bayesian(X_train,Y_train,datos,inputs,priori_x_grids,priori_y_grids) # Deficion del modelo por variable y por fold
#---------------------------------------------------------------------------------------------------
#------------------------Save del modelo, datos de testeo y entrenamiento---------------------------
    modelos_fold[f'modelo_fold_{j}'] = model
    data_train_fold[f'train_fold_{j}'] = {'x_train':X_train, 'y_train':Y_train, 'y_mean':y_mean, 'y_std':y_std, 'x_mean':x_mean, 'x_std':x_std}
    data_test_fold[f'test_fold_{j}'] = {'x_test':X_test, 'y_test':y_test}
    j = j+1
  modelos[f'modelo_{datos}'] = modelos_fold
  data_train[f'data_train_{datos}'] = data_train_fold
  data_test[f'data_test_{datos}'] = data_test_fold
#---------------------------------------------------------------------------------------------------

Band_gap
Fold # 1 para el parametro Band_gap
(50, 11)
Fold # 2 para el parametro Band_gap
(50, 11)
Fold # 3 para el parametro Band_gap
(50, 11)
Fold # 4 para el parametro Band_gap
(51, 11)
Fold # 5 para el parametro Band_gap
(51, 11)


<ipython-input-21-0ed8acf1d31c>:26: RuntimeWarning: invalid value encountered in divide
  X_train = (x_train-x_mean)/(x_std)
<ipython-input-21-0ed8acf1d31c>:29: RuntimeWarning: divide by zero encountered in divide
  X_test = (x_test-x_mean)/(x_std)
<ipython-input-21-0ed8acf1d31c>:29: RuntimeWarning: invalid value encountered in divide
  X_test = (x_test-x_mean)/(x_std)


DeltaH
Fold # 1 para el parametro DeltaH
(50, 11)
Fold # 2 para el parametro DeltaH
(50, 11)
Fold # 3 para el parametro DeltaH
(50, 11)
Fold # 4 para el parametro DeltaH
(51, 11)
Fold # 5 para el parametro DeltaH
(51, 11)


<ipython-input-21-0ed8acf1d31c>:26: RuntimeWarning: invalid value encountered in divide
  X_train = (x_train-x_mean)/(x_std)
<ipython-input-21-0ed8acf1d31c>:29: RuntimeWarning: divide by zero encountered in divide
  X_test = (x_test-x_mean)/(x_std)
<ipython-input-21-0ed8acf1d31c>:29: RuntimeWarning: invalid value encountered in divide
  X_test = (x_test-x_mean)/(x_std)


DeltaL
Fold # 1 para el parametro DeltaL
(50, 11)
Fold # 2 para el parametro DeltaL
(50, 11)
Fold # 3 para el parametro DeltaL
(50, 11)
Fold # 4 para el parametro DeltaL
(51, 11)
Fold # 5 para el parametro DeltaL
(51, 11)


...

<ipython-input-21-0ed8acf1d31c>:26: RuntimeWarning: invalid value encountered in divide
  X_train = (x_train-x_mean)/(x_std)
<ipython-input-21-0ed8acf1d31c>:29: RuntimeWarning: divide by zero encountered in divide
  X_test = (x_test-x_mean)/(x_std)
<ipython-input-21-0ed8acf1d31c>:29: RuntimeWarning: invalid value encountered in divide
  X_test = (x_test-x_mean)/(x_std)


Jsc
Fold # 1 para el parametro Jsc
(50, 11)
Fold # 2 para el parametro Jsc
(50, 11)
Fold # 3 para el parametro Jsc
(50, 11)
Fold # 4 para el parametro Jsc
(51, 11)
Fold # 5 para el parametro Jsc
(51, 11)


<ipython-input-21-0ed8acf1d31c>:26: RuntimeWarning: invalid value encountered in divide
  X_train = (x_train-x_mean)/(x_std)
<ipython-input-21-0ed8acf1d31c>:29: RuntimeWarning: divide by zero encountered in divide
  X_test = (x_test-x_mean)/(x_std)
<ipython-input-21-0ed8acf1d31c>:29: RuntimeWarning: invalid value encountered in divide
  X_test = (x_test-x_mean)/(x_std)


FF
Fold # 1 para el parametro FF
(50, 11)
Fold # 2 para el parametro FF
(50, 11)
Fold # 3 para el parametro FF
(50, 11)
Fold # 4 para el parametro FF
(51, 11)
Fold # 5 para el parametro FF
(51, 11)


<ipython-input-21-0ed8acf1d31c>:26: RuntimeWarning: invalid value encountered in divide
  X_train = (x_train-x_mean)/(x_std)
<ipython-input-21-0ed8acf1d31c>:29: RuntimeWarning: divide by zero encountered in divide
  X_test = (x_test-x_mean)/(x_std)
<ipython-input-21-0ed8acf1d31c>:29: RuntimeWarning: invalid value encountered in divide
  X_test = (x_test-x_mean)/(x_std)


### INFERENCIA (ENTRENAMIENTO)
La etapa de entrenamiento, mas especificamente de inferencia bayesiana es realizada a travez de la funcion sample de la libreria Pymc.
Esta funcion permite generar un trazado de las distribuciones posteriores de los parametros del modelo, por medio de MCMC. Esta funcion recibe como parametetros:
- draws: Que indica el numero muestras extraidas o "dibujos" tomados de la distribucion.
- tune: Es el numero de muestras que toma la funcion antes de iniciar la inferencia para un mejor ajuste y convergencia.
- chains: Es el numero de cadenas que va inferir la funcion.

Por ultimo se guarda las distribuciones de los parametros inferidos.

In [ ]:
#-------------------------------Definicion de parametros de sampleo---------------------------
draws = 1000 # se generan 1000 draws por modelo
tune = 500 # se generan 500 muestras de tuneado
chains = 5 # se generan 5 cadenas
#--------------------------------------------------------------------------------------------
trazados_data = {} #save del trazado
for datos in data[data.columns[3:]]: # se genera la inferencia para cada variable
  print(datos)
  trazados_fold = {}
  for i in range(1,6): # se abarcan cada modelo construido para cada fold para cada variable
#----------------------Etapa de entrenamieno o Inferencia------------------------------------
    with modelos[f'modelo_{datos}'][f'modelo_fold_{i}']:
      trace = pm.sample(draws = draws, tune = tune, chains = chains, cores = 4) # se genera la inferencia por medio de pm.sample
#--------------------------------------------------------------------------------------------
#-------------------------Save de los modelos inferidos--------------------------------------
    trazados_fold[f'trace_fold{i}'] = trace #save de trazados por fold
  trazados_data[f'trace_{datos}'] = trazados_fold # save de trazados por variables
#--------------------------------------------------------------------------------------------

Band_gap


ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


ERROR:pymc.stats.convergence:There were 1 divergences after tuning. Increase `target_accept` or reparameterize.


DeltaH


DeltaL


...

Voc


Jsc


ERROR:pymc.stats.convergence:There were 2 divergences after tuning. Increase `target_accept` or reparameterize.


FF


ERROR:pymc.stats.convergence:There were 1 divergences after tuning. Increase `target_accept` or reparameterize.


- GUARDADO DE LOS MODELOS

In [ ]:
#----------------------------Construccion y guardado de archivos descargables----------------------------
ruta_drive = '/content/drive/MyDrive/Deep Learning/TG/'
with open(ruta_drive + 'data_train.pkl', 'wb') as f:
    pickle.dump(data_train, f)
with open(ruta_drive + 'data_test.pkl', 'wb') as f:
    pickle.dump(data_test, f)
with open(ruta_drive + 'trazados.pkl', 'wb') as f:
    pickle.dump(trazados_data, f)